In [ ]:
# |default_exp masking


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# |export
from scip_workflows.common import *


In [ ]:
from skimage import img_as_float32


In [ ]:
# |export
import zarr
from scip.masking import get_bounding_box, remove_regions_touching_border, threshold


In [ ]:
# |export
def plot_scip_segmentation(r, bbox_channel_index=0, smooth=1, border=True):
    z = zarr.open(r.meta_path)
    pixels = z[r.meta_zarr_idx].reshape(z.attrs["shape"][r.meta_zarr_idx])
    pixels = numpy.clip(pixels, a_min=0, a_max=4096)

    m = threshold.get_mask(
        dict(pixels=pixels), main_channel=bbox_channel_index, smooth=smooth
    )
    m = get_bounding_box(m)
    if border:
        m = remove_regions_touching_border(m, bbox_channel_index=bbox_channel_index)

    fig, axes = plt.subplots(2, len(pixels), dpi=150, squeeze=False)
    for i, (a, p) in enumerate(zip(m["mask"], pixels)):
        axes[0, i].imshow(a)
        axes[0, i].set_axis_off()
        axes[1, i].imshow(p)
        axes[1, i].set_axis_off()

    return m


In [ ]:
zarr_path, zarr_idx = (
    "/data/gent/vo/000/gvo00070/vsc42015/datasets/weizmann/EhV/v2/images/Ctrl_T7_R1_22.zarr",
    0,
)
z = zarr.open(zarr_path)


In [ ]:
r = pandas.Series({"meta_path": zarr_path, "meta_zarr_idx": zarr_idx})


In [ ]:
pixels = z[zarr_idx].reshape(z.attrs["shape"][zarr_idx])


In [ ]:
pixels = numpy.clip(pixels, a_min=0, a_max=4096).astype(float)
pixels = img_as_float32(pixels)


In [ ]:
fig, axes = plt.subplots(1, len(pixels), dpi=150, squeeze=False)
axes = axes.ravel()
for p, ax in zip(pixels, axes):
    ax.imshow(p)
    ax.set_axis_off()


In [ ]:
from skimage.filters import (
    gaussian,
    sobel,
    threshold_isodata,
    threshold_mean,
    threshold_otsu,
)
from skimage.morphology import (
    binary_closing,
    binary_dilation,
    closing,
    disk,
    label,
    remove_small_holes,
    remove_small_objects,
)


In [ ]:
m = plot_scip_segmentation(r, 4)


In [ ]:
m = plot_scip_segmentation(r, 4)


In [ ]:
_ = plt.hist(numpy.log(pixels[4].ravel()), bins=100)


In [ ]:
t = threshold_otsu(numpy.log(pixels[4]))


In [ ]:
plt.imshow(((numpy.e**t) < pixels[4]))


In [ ]:
t = threshold_otsu(pixels[4])


In [ ]:
plt.imshow(t < pixels[4])


In [ ]:
bbox = m["bbox"]
x = pixels[4, bbox[0] : bbox[2], bbox[1] : bbox[3]]

x = gaussian(x, sigma=1)
# x = sobel(x)
# x = closing(x, selem=disk(2))
x = threshold_mean(numpy.log(x)) < numpy.log(x)
x = binary_dilation(x, disk(2))
# x = binary_closing(x, disk(1))
# x = remove_small_holes(x, area_threshold=5)
# x = remove_small_objects(x, min_size=5)
x = label(x)
plt.imshow(x)


In [ ]:
from skimage.morphology import disk, white_tophat
from skimage.restoration import denoise_nl_means, estimate_sigma


In [ ]:
pixels2 = pixels.copy()


In [ ]:
plt.imshow(white_tophat(pixels[1], disk(10)))


In [ ]:
sigma = estimate_sigma(pixels[1])
pixels2[1] = denoise_nl_means(
    pixels[1], patch_size=10, patch_distance=5, sigma=sigma, h=sigma
)
plt.imshow(pixels2[1])


In [ ]:
from scip.segmentation import threshold, util


In [ ]:
m = threshold.get_mask(dict(pixels=pixels), main=True, main_channel=0)
m = util.get_bounding_box(m, bbox_channel=0)
m = threshold.get_mask(m, main=False, main_channel=0)
plt.imshow(m["mask"][1])


In [ ]:
plt.imshow(m["pixels"][0])


# White tophat

In [ ]:
from skimage.morphology import disk, white_tophat


In [ ]:
bbox = m["bbox"]
bboxed = pixels[:, bbox[0] : bbox[2], bbox[1] : bbox[3]]


In [ ]:
plt.imshow(white_tophat(bboxed[1], selem=disk(3)))


In [ ]:
plt.imshow(pixels[0])
